In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

# url = "https://www.motogp.com/en/gp-results/2022/QAT/MotoGP/RAC/Classification"
url = "https://en.wikipedia.org/wiki/2022_MotoGP_World_Championship"

# page = requests.get(url)
data = requests.get(url).text

# soup = BeautifulSoup(page.content, "html.parser")
soup = BeautifulSoup(data, "html.parser")


In [2]:
dfs = pd.read_html(url, attrs = {'class': 'wikitable'},  flavor='bs4', thousands ='.')

In [3]:
df = dfs[5]
df.head()

,Pos.,Rider,Bike,Team,QAT,INA,ARG,AME,POR,SPA,...,GBR,AUT,RSM,ARA,JPN,THA,AUS,MAL,VAL,Pts
0,1,Francesco Bagnaia,Ducati,Ducati Lenovo Team,Ret,15,5,5,8,1P F,...,1,1,1,2P,Ret,3,3,1,9,265
1,2,Fabio Quartararo,Yamaha,Monster Energy Yamaha MotoGP,9,2P F,8,7,1F,2,...,8,2,5,Ret,8,17,Ret,3,4,248
2,3,Enea Bastianini,Ducati,Gresini Racing MotoGP,1F,11,10,1F,Ret,8,...,4,RetP,2F,1,9,6,5,2,8,219
3,4,Aleix Espargaró,Aprilia,Aprilia Racing,4,9,1P F,11,3,3,...,9,6,6,3,16,11,9,10,Ret,212
4,5,Jack Miller,Ducati,Ducati Lenovo Team,Ret,4,14,3,Ret,5,...,3,3,18P,5,1F,2,Ret,6,Ret,189


In [4]:
df1 = df.iloc[:, :4]

In [5]:
df2 = df.iloc[:, 4:].replace(r'\D+', "", regex=True).replace(r'^\s*$', np.nan, regex=True).fillna(25).astype(int)

In [6]:
df3 = pd.concat([df1, df2], axis=1)

In [7]:
df4 = df3.drop(["Pos.", "Bike", "Team", "Pts"], axis=1).set_index("Rider").drop("Rider").T

In [8]:
df4.index.names = ['Track']

In [9]:
df5 = df4.reset_index()

del df5["Stefan Bradl"]
del df5["Tetsuta Nagashima"] # NEED TO FIND A FIX FOR WHEN RIDER IS GROUPED

In [11]:
points_csv = pd.read_csv("C:/Users/ckhan/Documents/coding_projects/scraping_websites/motogp/data/championship_points_mapping.csv")
points_map = dict(points_csv.values)

In [12]:
def pts_fn(x):
    if int(x) in points_map.keys():
        return points_map[int(x)]
    else:
        return 0

In [13]:
df6 = pd.DataFrame()

df6["Track"] = df5["Track"]

for rider in df5.columns[1:]:
    df6[f"{rider}_points"] = df5[rider].map(lambda x: pts_fn(x))
    df6[f"{rider}"] = df6[f"{rider}_points"].cumsum()
    del df6[f"{rider}_points"]



In [15]:
import plotly.express as px


fig1 = px.line(
                df5, 
                x=df5["Track"], 
                y=df5.columns[1:], 
                template="plotly_dark",
                labels={
                    "Track": "Track",
                    "value": "Position",
                    "variable": "Rider"
                    },
                title="MotoGp Rider Position 2022",
            )

fig1['layout']['yaxis']['autorange'] = "reversed"

# fig1.update_layout(height=800, width=2250)

fig1.show()


In [16]:
import plotly.express as px


fig2 = px.line(
                df6, 
                x=df6["Track"], 
                y=df6.columns[1:], 
                template="plotly_dark",
                labels={
                    "Track": "Track",
                    "value": "Points Total",
                    "variable": "Rider"
                    },
                title="MotoGp Cumulative Points 2022",
            )

# fig2.update_layout(height=800, width=2250)

fig2.show()


In [17]:
import plotly.offline as pyo

figs = [fig1, fig2]


dashboard = open("motogp2022_dashboard.html", 'w')
dashboard.write("<html><head></head><body>" + "\n")

add_js = True
for fig in figs:

    inner_html = pyo.plot(
        fig, include_plotlyjs=add_js, output_type='div'
    )

    dashboard.write(inner_html)
    add_js = False

dashboard.write("</body></html>" + "\n")

15

In [18]:
df5.to_csv("motogp-2022_race_finishes.csv", index=False)
df6.to_csv("motogp-2022_cumulative_points.csv", index=False)

In [101]:
import requests

url = 'https://resources.motogp.com/files/results/2022/QAT/MotoGP/RAC/Classification.pdf'
response = requests.get(url, stream=True)

In [103]:
with open("test.pdf", "wb") as f:
    f.write(response.content)

In [112]:
from PyPDF2 import PdfReader

reader = PdfReader(r"C:\Users\ckhan\Documents\coding_projects\scraping_websites\test.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"


In [123]:
with open ('file_name.txt', 'w') as file:  
    file.write(text)

In [136]:
for i in range(7,32):
    print(text.splitlines()[i])

23 DUCATI Enea BASTIANINI ITA 1 42'13.198168.2 25 Gresini Racing MotoGP
33 KTM Brad BINDER RSA 2 42'13.544168.1 0.346 20 Red Bull KTM Factory Racing
44 HONDA Pol ESPARGARO SPA 3 42'14.549168.1 1.351 16 Repsol Honda Team
41 APRILIA Aleix ESPARGARO SPA 4 42'15.440168.0 2.242 13 Aprilia Racing
93 HONDA Marc MARQUEZ SPA 5 42'17.297167.9 4.099 11 Repsol Honda Team
36 SUZUKI Joan MIR SPA 6 42'18.041167.8 4.843 10 Team SUZUKI ECSTAR
42 SUZUKI Alex RINS SPA 7 42'22.008167.6 8.810 9 Team SUZUKI ECSTAR
5 DUCATI Johann ZARCO FRA 8 42'23.734167.5 10.536 8 Pramac Racing
20 YAMAHA Fabio QUARTARARO FRA 9 42'23.741167.5 10.543 7 Monster Energy Yamaha MotoGP
30 HONDA Takaaki NAKAGAMI JPN 10 42'28.165167.2 14.967 6 LCR Honda IDEMITSU
21 YAMAHA Franco MORBIDELLI ITA 11 42'29.910167.1 16.712 5 Monster Energy Yamaha MotoGP
12 APRILIA Maverick VIÑALES SPA 12 42'36.414166.6 23.216 4 Aprilia Racing
10 DUCATI Luca MARINI ITA 13 42'40.481166.4 27.283 3 Mooney VR46 Racing Team
4 YAMAHA Andrea DOVIZIOSO ITA 14 42

In [13]:
df7 = pd.read_csv("C:/Users/ckhan/Documents/coding_projects/scraping_websites/motogp/data/motogp-fantasy-2022.csv")

In [15]:
import plotly.express as px


fig2 = px.line(
                df7, 
                x=df7["Race"], 
                y=df7.columns[1:], 
                template="plotly_dark",
                labels={
                    "Track": "Track",
                    "value": "Points",
                    "variable": "Rider"
                    },
                title="MotoGp Fantasy Points 2022",
            )

# fig2.update_layout(height=800, width=2250)

fig2.show()

In [21]:
df8 = pd.DataFrame()

df8["Race"] = df7["Race"]

for rider in df7.columns[1:]:
    # df8[f"{rider}_points"] = df7[rider].map(lambda x: pts_fn(x))
    df8[f"{rider}"] = df7[rider].cumsum()
    # del df8[f"{rider}_points"]

In [22]:
import plotly.express as px


fig2 = px.line(
                df8, 
                x=df8["Race"], 
                y=df8.columns[1:], 
                template="plotly_dark",
                labels={
                    "Track": "Track",
                    "value": "Points",
                    "variable": "Rider"
                    },
                title="MotoGp Cumulative Fantasy Points 2022",
            )

# fig2.update_layout(height=800, width=2250)

fig2.show()